## AIF_sprint24_seq2seq
1.この課題の目的   
RNNの活用例を知る   
慣れていないフレームワークとの関わり方を知る   

【目的としないこと】  
手法の細かい部分への理解   
新たなフレームワークへの詳しい理解  

In [1]:
'''Sequence to sequence example in Keras (character-level).
This script demonstrates how to implement a basic character-level
sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.
# Summary of the algorithm
- We start with input sequences from a domain (e.g. English sentences)
    and corresponding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    Is uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.
# Data download
English to French sentence pairs.
http://www.manythings.org/anki/fra-eng.zip
Lots of neat sentence pairs datasets can be found at:
http://www.manythings.org/anki/
# References
- Sequence to Sequence Learning with Neural Networks
    https://arxiv.org/abs/1409.3215
- Learning Phrase Representations using
    RNN Encoder-Decoder for Statistical Machine Translation
    https://arxiv.org/abs/1406.1078
'''
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
from google.colab import files
uploaded = files.upload()

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

Using TensorFlow backend.


Saving fra.txt to fra.txt
Number of samples: 10000
Number of unique input tokens: 69
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59
Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 26s 3ms/step - loss: 0.9183 - val_loss: 0.9345
Epoch 2/100
8000/8000 [==============================] - 23s 3ms/step - loss: 0.7319 - val_loss: 0.7925
Epoch 3/100
8000/8000 [==============================] - 23s 3ms/step - loss: 0.6230 - val_loss: 0.6986
Epoch 4/100
8000/8000 [==============================] - 23s 3ms/step - loss: 0.5654 - val_loss: 0.6446
Epoch 5/100
8000/8000 [==============================] - 23s 3ms/step - loss: 0.5233 - val_loss: 0.6086
Epoch 6/100
8000/8000 [==============================] - 23s 3ms/step - loss: 0.4923 - val_loss: 0.5709
Epoch 7/100
8000/8000 [==============================] - 23s 3ms/step - loss: 0.4666 - val_loss: 0.5564
Epoch 8/100
8000/8000 [==========

/usr/local/lib/python3.6/dist-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


-
Input sentence: Go.
Decoded sentence: Va !

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Wow!
Decoded sentence: Ça alors !

-
Input sentence: Fire!
Decoded sentence: Au feu !

-
Input sentence: Help!
Decoded sentence: À l'aide !

-
Input sentence: Jump.
Decoded sentence: Saute.

-
Input sentence: Stop!
Decoded sentence: Arrête-toi !

-
Input sentence: Stop!
Decoded sentence: Arrête-toi !

-
Input sentence: Stop!
Decoded sentence: Arrête-toi !

-
Input sentence: Wait!
Decoded sentence: Attendez !

-
Input sentence: Wait!
Decoded sentence: Attendez !

-
Input sentence: Go on.
Decoded sentence: Poursuivens, je me place.

-
Input sentence: Go on.
Decoded sentence: Poursuivens, je me place.

-
Input sentence: Go on.
Decoded sentence: Poursuivens, je me place.

-
Input sentence: Hello!
Decoded sentence: Salut !

-
Input sentence: Hello!
Decoded sentence: Salut !

-
Inpu

## コードリーディング

### 説明（サンプルコードの各部分がどういった役割かを読み取る）
43から49行目 : ライブラリのimport    
51から56行目 : ハイパーパラメータの設定      
59から116行目 : 文字のシーケンスのベクトル化      
119から123行目 : seq2seqモデルの構築（エンコーダ）     
130から134行目 : seq2seqモデルの構築（デコーダ）      
138から147行目 : モデルのコンパイル設定      
158から169行目 : デコーダの実装     
179から214行目 : モデルの予測に基づいて次の文字をサンプリングする関数      
217から224行目 : テキスト生成ループ    

### Seq2Seqは何をしているか
Seq2Seqはエンコーダとデコーダの2つのモジュールで構成されている。     
エンコーダはRNNを利用して、時系列データを隠れ状態ベクトルに変換する。ここでは、入力された単語から、次に出現する単語を確率的に選択する。RNNとしてはLSTMを利用しており、エンコーダが出力するベクトル、すなわちLSTMレイヤの最後の隠れ状態には、入力文章を翻訳するために必要な情報がエンコードされている。      
エンコードでは、最後の隠れ状態は固定長のベクトルに変換している。また、最後の隠れ状態がエンコーダとデコーダの架け橋となっている。      
デコーダでも、RNNを利用してエンコーダと同様のモデルであり、エンコードされた情報を文字に変換することによって、機械翻訳を行なっている。    


## イメージキャプショニングをシェルで実行

andoutakaakinoMacBook-ea:image_captioning andoutakaaki$ python sample.py --image='png/example.png'
Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /Users/andoutakaaki/.torch/models/resnet152-b121ed2d.pth
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 241530880/241530880 [00:04<00:00, 56435250.05it/s]
<start> a group of giraffes standing next to each other . <end>

## Kerasで動かしたい場合はどうするか
説明

PyTorchによる実装を動かしたが、何らかの理由からKerasで動かしたい状況が考えられる。どういった手順を踏むことになるか調査し、できるだけ詳しく説明せよ。

特に今回はPyTorchのための学習済みの重みをKerasで使えるようにしたいので、その点についてはしっかりと触れること。

## 説明課題

### 今回使用したSeq2Seqの実装を使い日本語と英語の翻訳を行いたい場合はどのような手順を踏むか
・まず、リンクでつながっている例文同士が、互いに対訳関係にある例文が掲載されている日英の対訳データを入手し、対訳ペアのリストを作成する。       
・次に、データの整形を行う。まず、日本語の単語の分割をする必要がある。そのために、mecabなどを利用して、形態素解析を行う。    
・その上で、単語の正規化（大文字小文字）、Stemming処理（活用形の処理）を行なって、テキストを単語IDに変換する。     
・次に、テキストからコンテキストとターゲットを作り、one-hot表現に変換するといった手順により、前処理を行う。

### 機械翻訳のための発展的手法にはどういったものがあるか
・機械翻訳の流れでみると、「ルールベースの翻訳」から「用例ベースの翻訳」へ、そして「統計ベースの翻訳」へと移り変わってきた。現在では、これらの従来の技術に代わり、ニューラル翻訳が多くの注目を集めている。Google翻訳では、2016年からニューラル翻訳が使われている。この機械翻訳システムは、GNMT（Google Neural Mashine Translation）と呼ばれ、翻訳精度向上のために多くの改良がなされている。             
・また、並列化処理ができないRNNの欠点に対して、現在、RNNを取り除く研究（もしくは並列計算可能なRNNの研究）が活発に行われている。その中で有名なものに、Transformerと呼ばれるモデルがある。Transformerでは、Self-Attentionというテクニックが使われているのが特徴的である。      
・また、DeepMindのチームによって行われた研究に、NMT（Neural Turing Machine）がある。NMTは、RNNの外側に情報を記憶しておくためのメモリ機能を配置し、Attention（２つの時系列データ間の対応関係をデータから学習する）を使って、そのメモリから必要な情報を読み書きさせる方法。

### イメージキャプショニングとは逆に文章から画像を生成する手法にはどういったものがあるか
・Microsoftは、Generative Adversarial Network (GAN: 敵対的生成ネットワーク)を利用して、文章から画像を生成する人工知能技術を開発した。GANはGeneratorとDiscriminatorという別々の学習モデルを用いる。Generatorはできるだけ本物（オリジナル）に近い画像を生成し、Discriminatorはそれが本物の画像か否かを判定するような構造をしており、両方を競わせることで、より本物に近い画像などを生成することを目的としている。    
・Microsoftは、GANを改良したAttentional GAN (AttnGAN)を開発しており、AttnGANでは、入力されたキャプションを個々の単語に分割し、それらの単語を画像の特定の領域とマッチングさせることで、人間のような描画を実現している。

### 異なるフレームワークに実装を移す場合にはどういった部分を気をつけるか
・例えば、ディープラーニングで人気の高いKerasとPyTorchを比較してみる。    
・Kerasは、一般的に使用されているレイヤとオペレーションをきちんとしたレゴサイズのビルディングブロックにまとめ、ディープラーニングの複雑さを抽象化し、コードの記述が簡潔で、使いやすい。一方、PyTorchは、ユーザーにカスタムレイヤーを書いたり数値最適化タスクのフードの下を見たりする自由を与えており、ネットワークの詳細設計をすることができる。        
・すなわち、Kerasは読みやすくて簡潔であり、実装の詳細をスキップしながら最初のエンドツーエンドのディープラーニングモデルをより速く構築することを可能にする。ただし、他のライブラリに比べて細かい設定が隠匿されているため、デバッグの際、問題の原因となる正確な行を突き止めるのが難しくなる。    
・一方で、PyTorchは、より冗長なフレームワークであるため、スクリプトの実行を1行ずつ追跡することができる。    
・したがって、実装の上では、Kerasで比べ、PyTorchは自由度が高い分、ネットワークの詳細設計のためのモデル構築を行う必要がある。     